In [33]:
# Load data with spam classification based on simple rules:
import pandas as pd 
folder='../data/'
file_name='text_preprocessed_Spamlabeled.csv'
df=pd.read_csv(folder+file_name, sep="|", encoding ='utf-8', index_col='Unnamed: 0')
labels = df["Spam-index"]
print (df.shape[0])
df.head()

129437


,date,text,link_title,link_description,unigrams,bigrams,Spam-index
0,2014-11-20,Бузык барин булдирген,NaN,NaN,"['бузык', 'барин', 'булдирген']","['бузык_барин', 'барин_булдирген']",0
1,2014-10-28,Алтын куз,NaN,NaN,"['алтын', 'куз']",['алтын_куз'],0
2,2014-07-14,Биз бакиттымыз,NaN,NaN,"['биз', 'бакиттымыза']",['биз_бакиттымыза'],0
3,2016-09-05,Молодцы!!!Красиво!!!,NaN,NaN,"['молодец', 'красиво']",['молодец_красиво'],0
4,2019-07-03,Самый Красавец Кот,NaN,NaN,"['самый', 'красавец', 'кот']","['самый_красавец', 'красавец_кот']",0


In [34]:
print ('spam (1) and not-spam (0) counts: ')
print (labels.value_counts())

spam (1) and not-spam (0) counts: 
0    90523
1    38914
Name: Spam-index, dtype: int64


Load model rubert-tiny2 

In [4]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel 
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational-v1", model_max_length=512)
model = AutoModel.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational-v1", num_labels=2)

Some weights of the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational-v1 were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from logitboost import LogitBoost
from sklearn.metrics import accuracy_score, classification_report

In [6]:
# === First run only - 1) Allocate 1000 for validation. 2) Divide into train, test 3) Save and then only load the saved subsamples

and get embeddings (embeddings were previously generated, so here we load them from X_train_test_emb.csv, X_valid_emb.csv)

In [315]:
# Allocate 1000 for validation
# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(df_emb_, labels, test_size = 1000, stratify = labels, random_state = 1)

After the initial split we train models and then edit train classes manually- some of those classes (predicted by simple rules) that are inconsistent with classes predicted by the model (LogitBoost based on rubert-tiny2 embeddings).  
Here we provide the next step - we train models after manual editing

=== Load train ====

In [46]:
import pandas as pd
res_folder='../data/data_for_spam_filtering/'

df_train=pd.read_csv(res_folder+'train-test_edited.csv', sep="|", encoding ='utf-8', index_col='Column1')[['text', 'valid-label']]
y_train=df_train['valid-label']
df_X_train=pd.read_csv(res_folder+'X_train_test_emb.csv', sep='|', encoding='utf-8')
df_X_train.drop('Unnamed: 0', axis=1, inplace=True)
X_train=df_X_train.to_numpy()


Load valid data set:

In [35]:
df_valid=pd.read_csv(res_folder+'valid.csv', sep="|", encoding ='utf-8')
df_valid.drop('Column1', axis=1, inplace=True)
print ("label - predicted by simple rules, valid-label - classes by expert")
df_valid.head()


label - predicted by simple rules, valid-label - classes by expert


,text,label,valid-label
0,МиниТест «Какой телефон подходит тебе? » показ...,1,1
1,Красивые правда?,0,0
2,Посвещается всем тётям !!,0,0
3,НОВЫЙ ТАНЦЕВАЛЬНЫЙ СЕЗОН 2019-2020 УЖЕ ЗДЕСЬ 💣...,1,1
4,__шик лофери __еколак _-підошва резина прошиті...,1,1


In [36]:
y_valid=df_valid['valid-label'].tolist()
y_simple_rules=df_valid['label'].tolist()

Validate classification based on simple rules

In [38]:
report_valid = classification_report(y_valid, y_simple_rules)
print('Valid\n%s' % report_valid)

Valid
              precision    recall  f1-score   support

           0       0.91      0.99      0.95       640
           1       0.98      0.82      0.89       360

    accuracy                           0.93      1000
   macro avg       0.94      0.91      0.92      1000
weighted avg       0.93      0.93      0.93      1000



In [37]:
df_X_valid=pd.read_csv(res_folder+'X_valid_emb.csv', sep='|', encoding='utf-8')
df_X_valid.drop('Unnamed: 0', axis=1, inplace=True)
X_valid=df_X_valid.to_numpy()

In [8]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(
    class_weight='balanced', classes=np.unique(y_valid), y=y_valid)
class_weight

array([0.78125   , 1.38888889])

Logit regression

In [ ]:
# lr_clf = LogisticRegression(C=26.31586, class_weight={0:class_weight[0], 1:class_weight[1]})     #(C=grid_search.best_params_['C'])   
parameters = {'C': np.linspace(0.0001, 10, 50), "penalty":["l1","l2"]}  #high C means "Trust this training data a lot", while a low value says "This data may not be fully representative of the real world data, so if it's telling you to make a parameter really large, don't listen to it"
grid_search = GridSearchCV(LogisticRegression(class_weight={0:class_weight[0], 1:class_weight[1]}), parameters, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)


In [ ]:
print("Best parameters from gridsearch: {}".format(grid_search.best_params_))
print("CV score=%0.3f" % grid_search.best_score_)
cv_resultsl = grid_search.cv_results_


In [63]:
# prediction for train dataset
y_pred_train=(grid_search.predict_proba(X_train)[:,1] >= 0.5).astype(int)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train)) 
print("Precision:",metrics.precision_score(y_train, y_pred_train)) 
print("Recall:",metrics.recall_score(y_train, y_pred_train))

Accuracy: 0.9258469132726551
Precision: 0.8654765699713102
Recall: 0.9131312160668813


In [66]:
y_pred_valid=(grid_search.predict_proba(X_valid)[:,1] >= 0.5).astype(int)
report_valid = classification_report(y_valid, y_pred_valid)
print('Valid\n%s' % report_valid)

Valid
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       640
           1       0.90      0.85      0.88       360

    accuracy                           0.91      1000
   macro avg       0.91      0.90      0.90      1000
weighted avg       0.91      0.91      0.91      1000



LogitBoost

In [ ]:
steps = [('LogitBoost', LogitBoost())]
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps)
parameteres = {'LogitBoost__n_estimators':range(10,100,10)}
grid_boost = GridSearchCV(pipeline, param_grid=parameteres, cv=5)
grid_boost.fit(X_train, y_train)


In [69]:
y_pred_train = (grid_boost.predict_proba(X_train)[:,1] >= 0.5).astype(int)
accuracy_train = accuracy_score(y_train, y_pred_train)
print('Training accuracy: %.4f' % accuracy_train)
report_train = classification_report(y_train, y_pred_train)
print('Training\n%s' % report_train)

Training accuracy: 0.9442
Training
              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96     86811
         1.0       0.94      0.88      0.91     41626

    accuracy                           0.94    128437
   macro avg       0.94      0.93      0.94    128437
weighted avg       0.94      0.94      0.94    128437



In [70]:
y_pred_valid = (grid_boost.predict_proba(X_valid)[:,1] >= 0.5).astype(int)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)

print('Valid accuracy: ', accuracy_valid)
report_valid = classification_report(y_valid, y_pred_valid)
print('Valid\n%s' % report_valid)

Valid accuracy:  0.917
Valid
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       640
           1       0.96      0.80      0.87       360

    accuracy                           0.92      1000
   macro avg       0.93      0.89      0.91      1000
weighted avg       0.92      0.92      0.92      1000



SVM

In [12]:
# parameteres = {'C': [1], 'gamma': [1],'kernel': ['rbf']}
# grid_svm = GridSearchCV(SVC(class_weight={0:class_weight[0], 1:class_weight[1]}, probability=True), parameteres , cv=5, scoring='f1_macro')
grid_svm.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=SVC(class_weight={0: 0.78125, 1: 1.3888888888888888},
                           probability=True),
             param_grid={'C': [1], 'gamma': [1], 'kernel': ['rbf']},
             scoring='f1_macro')

In [47]:
y_pred_train = (grid_svm.predict_proba(X_train)[:,1] >= 0.5).astype(int)
accuracy_train = accuracy_score(y_train, y_pred_train)
print('Training accuracy: %.4f' % accuracy_train)
report_train = classification_report(y_train, y_pred_train)
print('Training\n%s' % report_train)


Training accuracy: 0.9560
Training
              precision    recall  f1-score   support

         0.0       0.96      0.97      0.97     86811
         1.0       0.94      0.92      0.93     41626

    accuracy                           0.96    128437
   macro avg       0.95      0.95      0.95    128437
weighted avg       0.96      0.96      0.96    128437



In [14]:
y_pred_valid = (grid_svm.predict_proba(X_valid)[:,1] >= 0.5).astype(int)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)

print('Valid accuracy: ', accuracy_valid)
report_valid = classification_report(y_valid, y_pred_valid)
print('Valid\n%s' % report_valid)

Valid accuracy:  0.943
Valid
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       640
           1       0.97      0.87      0.92       360

    accuracy                           0.94      1000
   macro avg       0.95      0.93      0.94      1000
weighted avg       0.94      0.94      0.94      1000



The best model is SVM. Calculate the number of spam posts according to the SVM model:

In [49]:
print ('number of spam posts:',sum(y_pred_train)+sum(y_pred_valid))

number of spam posts: 41191
